In [ ]:
"""
It is assumed that the COMA library is in the coma_lib directory:

https://github.com/anuragranj/coma/tree/master/lib
"""

import sys
sys.path.append('./coma_lib')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import cPickle as pickle
import glob
import menpo3d.io as m3io
from menpo.shape import TriMesh, PointCloud
from os.path import join

import pyface.qt
import numpy as np

import mesh_sampling
from psbody.mesh import Mesh

from compute_operators import *
from utils import laplacian

%matplotlib qt

## Import a mesh

In [ ]:
DATA_PATH = '../data/'
TYPE = 'template'
COARSENING = 'sampling_4'

template = Mesh(filename=join(DATA_PATH, TYPE, 'template_triangulated.obj'))

## Compute mesh sampling transforms from the COMA code

In [ ]:
# sampling_factors = 1.0 / np.array([2] * 8)
# pooling_data = compute_downsampling_transforms(template, sampling_factors)

## Compute mesh sampling transforms from the decimated meshes

In [ ]:
# Import decimated meshes.
downsampled_paths = glob.glob(join(DATA_PATH, TYPE, COARSENING, 'downsampled_meshes', '*.obj'))
downsampled_paths.sort()
down_meshes = [template] + [Mesh(filename=path) for path in downsampled_paths]

# Specify the sampling factors for clarity.
sampling_factors = 1.0 / np.array([4, 4, 2, 2])
pooling_data = compute_transforms_from_downsampled_meshes(down_meshes, template, sampling_factors)

## Visualize downsampled meshes

In [ ]:
down_meshes = pooling_data['down_meshes']
for down_mesh in down_meshes:
    TriMesh(down_mesh.v, down_mesh.f).view(colour='c')
    print(len(down_mesh.v))

## Export meshes

In [ ]:
# for down_mesh in down_meshes:
#     m3io.export_mesh(TriMesh(down_mesh.v, down_mesh.f), str(ii) + '.obj')

## Export the pooling data

In [ ]:
D = pooling_data['down_transforms']
U = pooling_data['up_transforms']

transforms = {'down_transforms' : D, 'up_transforms' : U}

pickle.dump(pooling_data, open(join(DATA_PATH, TYPE, COARSENING, 'operators', 'pooling.p'), 'wb'))
pickle.dump(transforms, open(join(DATA_PATH, TYPE, COARSENING, 'operators', 'transforms.p'), 'wb'))

## Compute the Laplacian of the adjacency matrix

In [ ]:
L = compute_laplacian(pooling_data['adj_matrices'])
spec = {'laplacian' : L, 'adjacency': pooling_data['adj_matrices']}

pickle.dump(spec, open(join(DATA_PATH, TYPE, COARSENING, 'operators', 'spectral_LA.p'), 'wb'))